In [53]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow import keras
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix

In [54]:
DATA_DIR = "data_all"
IMG_SIZE = 128

fruit_classes = ["ananas", "banane", "tomate", "papaye", "non_fruit"]
maturity_classes = ["pas_mur", "mur", "trop_mur"]

X = []
y_fruit = []
y_maturity = []


for fruit in fruit_classes:
    fruit_path = os.path.join(DATA_DIR, fruit)
    if not os.path.isdir(fruit_path):
        continue

    if fruit == "non_fruit":
        files = [f for f in os.listdir(fruit_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        for filename in files:
            img_path = os.path.join(fruit_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0

            X.append(img)
            y_fruit.append(fruit)
            y_maturity.append(3)
        continue

    for subdir in os.listdir(fruit_path):
        subdir_path = os.path.join(fruit_path, subdir)
        if not os.path.isdir(subdir_path):
            continue

        files = [f for f in os.listdir(subdir_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        for filename in files:
            img_path = os.path.join(subdir_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0

            X.append(img)
            y_fruit.append(fruit)

            if subdir in maturity_classes:
                y_maturity.append(maturity_classes.index(subdir))
            else:
                y_maturity.append(3)


X = np.array(X, dtype=np.float32)
fruit_indices = [fruit_classes.index(f) for f in y_fruit]
y_fruit = to_categorical(fruit_indices, num_classes=len(fruit_classes))

y_maturity = to_categorical(y_maturity, num_classes=len(maturity_classes)+1)
maturity_mask = (np.argmax(y_maturity, axis=1) != 3).astype("float32")

X_train, X_test, y_fruit_train, y_fruit_test, y_maturity_train, y_maturity_test, mask_train, mask_test = train_test_split(
    X, y_fruit, y_maturity, maturity_mask, test_size=0.2, random_state=42
)

print("Train :", X_train.shape, y_fruit_train.shape, y_maturity_train.shape)
print("Test  :", X_test.shape, y_fruit_test.shape, y_maturity_test.shape)


Train : (3802, 128, 128, 3) (3802, 5) (3802, 4)
Test  : (951, 128, 128, 3) (951, 5) (951, 4)


In [62]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

IMG_SIZE = 128
num_fruit_classes = 5
num_maturity_classes = 3


input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

fruit_output = Dense(num_fruit_classes, activation='softmax', name='fruit_output')(x)

maturity_output = Dense(num_maturity_classes + 1, activation='softmax', name='maturity_output')(x)

model = Model(inputs=input_layer, outputs=[fruit_output, maturity_output])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'fruit_output': 'categorical_crossentropy',
        'maturity_output': 'categorical_crossentropy'
    },
    metrics={
        'fruit_output': 'accuracy',
        'maturity_output': 'accuracy'
    }
)

model.summary()


early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath="fruit_maturity_detector.keras",
    monitor="val_loss",
    save_best_only=True,
    mode="min",
    verbose=1
)

history = model.fit(
    X_train,
    {
        'fruit_output': y_fruit_train,
        'maturity_output': y_maturity_train
    },
    validation_data=(
        X_test,
        {
            'fruit_output': y_fruit_test,
            'maturity_output': y_maturity_test
        }
    ),
    epochs=15,
    batch_size=32,
    callbacks=[checkpoint]  
)


# model.save("fruit_maturity_detector.keras")

print("Modèle sauvegardé dans fruit_maturity_detector.keras")


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 128, 128,  │        896 │ input_layer_7[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_21    │ (None, 64, 64,    │          0 │ conv2d_21[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 64, 64,    │     18,496 │ max_pooling2d_21… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_22    │ (None, 32, 32,    │          0 │ conv2d_22[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 32, 32,    │     73,856 │ max_pooling2d_22… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_23    │ (None, 16, 16,    │          0 │ conv2d_23[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_7 (Flatten) │ (None, 32768)     │          0 │ max_pooling2d_23… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 256)       │  8,388,864 │ flatten_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 256)       │          0 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fruit_output        │ (None, 5)         │      1,285 │ dropout_7[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maturity_output     │ (None, 4)         │      1,028 │ dropout_7[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,484,425 (32.37 MB)

 Trainable params: 8,484,425 (32.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - fruit_output_accuracy: 0.8732 - fruit_output_loss: 0.4113 - loss: 1.2245 - maturity_output_accuracy: 0.6610 - maturity_output_loss: 0.8132
Epoch 1: val_loss improved from None to 0.39372, saving model to fruit_maturity_detector.keras
119/119 ━━━━━━━━━━━━━━━━━━━━ 39s 312ms/step - fruit_output_accuracy: 0.9477 - fruit_output_loss: 0.1915 - loss: 0.7823 - maturity_output_accuracy: 0.7733 - maturity_output_loss: 0.5908 - val_fruit_output_accuracy: 0.9853 - val_fruit_output_loss: 0.0502 - val_loss: 0.3937 - val_maturity_output_accuracy: 0.8759 - val_maturity_output_loss: 0.3457
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - fruit_output_accuracy: 0.9831 - fruit_output_loss: 0.0606 - loss: 0.4021 - maturity_output_accuracy: 0.8777 - maturity_output_loss: 0.3415
Epoch 2: val_loss improved from 0.39372 to 0.26354, saving model to fruit_maturity_detector.keras
119/119 ━━━━━━━━━━━━━━━━━━━━ 38s 320ms/step - fruit_output_accuracy: 0.

In [63]:
results = model.evaluate(
    X_test,
    {
        'fruit_output': y_fruit_test,
        'maturity_output': y_maturity_test
    },
    batch_size=32
)

print("Résultats de l'évaluation :", results)


30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - fruit_output_accuracy: 0.9979 - fruit_output_loss: 0.0059 - loss: 0.2162 - maturity_output_accuracy: 0.9359 - maturity_output_loss: 0.2086
Résultats de l'évaluation : [0.21615181863307953, 0.005924052558839321, 0.20861075818538666, 0.9978969693183899, 0.9358569979667664]


In [64]:
y_fruit_test_labels = np.argmax(y_fruit_test, axis=1)
y_fruit_pred_labels = np.argmax(model.predict(X_test)[0], axis=1)

print(confusion_matrix(y_fruit_test_labels, y_fruit_pred_labels))
print(classification_report(y_fruit_test_labels, y_fruit_pred_labels, target_names=fruit_classes))


30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step
[[ 15   0   0   0   0]
 [  0 493   0   1   0]
 [  0   0 392   1   0]
 [  0   0   0  47   0]
 [  0   0   0   0   2]]
              precision    recall  f1-score   support

      ananas       1.00      1.00      1.00        15
      banane       1.00      1.00      1.00       494
      tomate       1.00      1.00      1.00       393
      papaye       0.96      1.00      0.98        47
   non_fruit       1.00      1.00      1.00         2

    accuracy                           1.00       951
   macro avg       0.99      1.00      1.00       951
weighted avg       1.00      1.00      1.00       951



TEST 1

In [65]:
img = cv2.imread("banane_mur.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (128, 128))
img = img / 255.0
img = np.expand_dims(img, axis=0)

pred_fruit, pred_maturity = model.predict(img)

fruit_idx = np.argmax(pred_fruit)
maturity_idx = np.argmax(pred_maturity)

print("Fruit prédit :", fruit_classes[fruit_idx])
print("Maturité prédite :", maturity_classes[maturity_idx])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Fruit prédit : banane
Maturité prédite : mur


TEST 2

In [66]:

model_path = "fruit_maturity_detector.keras"
IMG_SIZE = 128

fruit_classes = ["ananas", "banane", "tomate", "papaye", "non_fruit"]
maturity_classes = ["pas_mur", "mur", "trop_mur"]

model = load_model(model_path)

def predict_image(image_path):
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Erreur : impossible de lire l'image {image_path}")
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    pred_fruit, pred_maturity = model.predict(img)
    fruit_idx = np.argmax(pred_fruit)
    maturity_idx = np.argmax(pred_maturity)

    if fruit_classes[fruit_idx] == "non_fruit":
        print(f"Image : {image_path} → Ce n'est pas un fruit.")
    else:
        print(f"Image : {image_path} → Fruit : {fruit_classes[fruit_idx]}, Maturité : {maturity_classes[maturity_idx]}")

predict_image("tomate_mur.jpg")
predict_image("banane_mur.jpg")
predict_image("non_fruit3.png")
predict_image("non_fruit1.png")
predict_image("7_dos.jpg")

predict_image("tomate_mur1.png")
# predict_image("papaye_mur.jpg")
predict_image("pas_mur.png")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Image : tomate_mur.jpg → Fruit : tomate, Maturité : mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image : banane_mur.jpg → Fruit : banane, Maturité : mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image : non_fruit3.png → Ce n'est pas un fruit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image : non_fruit1.png → Ce n'est pas un fruit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image : 7_dos.jpg → Fruit : ananas, Maturité : mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image : tomate_mur1.png → Fruit : tomate, Maturité : trop_mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image : pas_mur.png → Fruit : banane, Maturité : trop_mur


In [60]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results = model("pas_mur.png")
results[0].show()



image 1/1 c:\Users\HP\Desktop\Rendu\Semestre2\C-DAT-900-ABJ-2-1-ecp-14\pas_mur.png: 448x640 1 banana, 110.3ms
Speed: 4.4ms preprocess, 110.3ms inference, 1.9ms postprocess per image at shape (1, 3, 448, 640)
